In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import mixed_precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tqdm import tqdm

# Mixed Precision 설정 (A100 GPU 최적화)
mixed_precision.set_global_policy('mixed_float16')

# 진행 상황 출력용 Callback 클래스
class TQDMProgressBar(Callback):
    def __init__(self, total_epochs):
        super().__init__()
        self.total_epochs = total_epochs

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_bar = tqdm(total=self.total_epochs, desc=f"Epoch {epoch + 1}/{self.total_epochs}", position=0, leave=False)

    def on_epoch_end(self, epoch, logs=None):
        self.epoch_bar.update(1)
        self.epoch_bar.close()

    def on_train_end(self, logs=None):
        if hasattr(self, 'batch_bar'):
            self.batch_bar.close()

# JSON 데이터 로드 함수
def load_data_from_json(json_file):
    X = []
    y = []
    # JSON 파일 읽기
    with open(json_file, 'r') as f:
        data = json.load(f)
        for frame in data:
            X.append(frame[:-1])  # 마지막 값을 제외하고 키포인트 데이터로 사용
            y.append(frame[-1])  # 마지막 값은 라벨로 사용
    return np.array(X), np.array(y)

# 데이터 경로
json_file = "/content/data_val.json"

# JSON 데이터 로드
X, y = load_data_from_json(json_file)

# 데이터 전처리: 레이블 원-핫 인코딩
y = to_categorical(y)

# 데이터 분할 (개선된 방식)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42, shuffle=True)

# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE).repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).shuffle(2000).batch(64).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)

# 모델 불러오기 또는 새로 생성
def load_model_with_path(model_path):
    if os.path.exists(model_path):
        return load_model(model_path)
    else:
        # MLP 모델 설계
        model = Sequential([
            Conv1D(128, kernel_size=3, activation='relu', input_shape=(12, 3), kernel_regularizer=l2(0.0004)),
            BatchNormalization(),

            Conv1D(64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.0004)),
            BatchNormalization(),

            Conv1D(32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.0004)),
            BatchNormalization(),

            Conv1D(16, kernel_size=3, activation='relu', kernel_regularizer=l2(0.0004)),
            BatchNormalization(),

            GlobalAveragePooling1D(),

            Dense(y_train.shape[1], activation='softmax', dtype='float32')
        ])

        # 모델 컴파일
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

# 체크포인트 콜백 설정
checkpoint = ModelCheckpoint(
    'best_model.keras',  # 저장할 파일 이름 (.keras 형식)
    monitor='val_accuracy',  # 모니터링할 지표
    mode='max',  # 최대화할 지표 (val_accuracy가 최대일 때 저장)
    save_best_only=True,  # 가장 좋은 모델만 저장
    verbose=1  # 저장 시 로그 출력
)

# 학습 진행 상황 출력
progress_bar = TQDMProgressBar(total_epochs=20)

# 모델 저장 파일 경로
model = load_model_with_path('/content/best_model.keras')

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링할 지표 (예: 'val_loss' 또는 'val_accuracy')
    patience=5,          # 개선되지 않은 에포크 수 (5번 연속으로 개선되지 않으면 중지)
    restore_best_weights=True,  # 가장 좋은 가중치를 복원
    verbose=1            # 중지 시 로그 출력
)

# 모델 학습
model.fit(
    train_dataset,
    steps_per_epoch=100,  # 전체 데이터를 학습하지 않고 일부 데이터로 학습
    validation_data=val_dataset,
    epochs=10,
    callbacks=[progress_bar, checkpoint, early_stopping],  # 조기 종료 콜백 추가
)

# 모델 평가
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")
